In [22]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from config import configure
import os
from langchain_community.document_loaders import PyMuPDFLoader
from pydantic import BaseModel, Field
from model.topic_content import Topicsummary
from crewai import Agent,Crew,Process,Task,LLM
from crewai.project import agent,crew,task,CrewBase
from langchain_ollama import OllamaLLM
import logging
import PyPDF2 as pdf
# from langchain_openai import ChatOpenAI
from langchain_community.chat_models.openai import ChatOpenAI



In [66]:
llm_model=LLM(model="ollama/deepseek-r1:7b", base_url="http://127.0.0.1:11434")

In [65]:
temp_dir = 'temp/'
logging = 'temp/logging/'

In [9]:

class Topicsummary(BaseModel):
    topic: str = Field(..., description="The topics that discussed in the pdf or given text")

In [ ]:
# llm_model = OllamaLLM(model="crewai-deepseek-r1",api_key="NA")

In [ ]:
def pdf_to_text(pdf_path):
    # Open the PDF file in read-binary mode
    with open(pdf_path, 'rb') as pdf_file:
        # Create a PdfReader object instead of PdfFileReader
        pdf_reader = pdf.PdfReader(pdf_file)

        # Initialize an empty string to store the text
        text = ''

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    # Write the extracted text to a text file
    # with open('/crewai_deepseek/temp/output.txt', 'w', encoding='utf-8') as txt_file:
    #     txt_file.write(text)
    return text

In [ ]:
text = pdf_to_text("/crewai_deepseek/temp/glue-dg.pdf")

In [69]:
# Agent 1: Concept Definition Agent
concept_definition_agent = Agent(
    role="Concept Definition Agent",
    goal="To create comprehensive data dictionaries, metadata repositories, and data governance documentation, ensuring data consistency and quality."
    "To help users understand the structure and usage of data within a system or document, enabling accurate data management, analysis, and reporting.",
    verbose=True,
    backstory=(
        "Developed to assist in data governance, data modeling, and data warehousing projects."
        "Initially used for creating glossaries and educational content, it has been refined to provide in-depth conceptual explanations for diverse fields."
        "you are in the creation of educational materials, technical documentation, and knowledge bases."
    ),llm=llm_model

)

In [47]:
# Task for Researcher Agent: Extract definitions
concept_definition_task = Task(
    description=(
        "Identify and define all key concepts presented in the document." 
        "For each concept, provide a detailed definition along with contextual examples or use cases that illustrate its application." 
        "Present each concept in the format: 'Concept Name: Detailed Definition with Contextual Examples'."
    ),
    expected_output=(
        "A list of key concepts, with each concept accompanied by a detailed definition and contextual examples or use cases."
    ),
    agent=concept_definition_agent,
    async_execution=True
)

In [70]:
definition_extract_crew = Crew(agents=[concept_definition_agent],
     tasks=[concept_definition_task])

In [71]:
definition_extract_crew.kickoff(inputs=text)

# Agent: Concept Definition Agent
## Task: Identify and define all key concepts presented in the document.For each concept, provide a detailed definition along with contextual examples or use cases that illustrate its application.Present each concept in the format: 'Concept Name: Detailed Definition with Contextual Examples'.


🖇 AgentOps: Could not end session - no sessions detected




# Agent: Concept Definition Agent
## Final Answer: 
1. **Data Governance Framework**  
   - **Definition**: A structured approach to managing data policies, standards, and processes within an organization to ensure compliance, consistency, and quality of data assets.  
   - **Contextual Example**: Organizations like XYZ utilize Data Governance Frameworks to implement data quality policies that prevent errors and ensure data usability across departments.

2. **Data Quality Scorecard**  
   - **Definition**: A tool used to monitor and evaluate the quality of data within an organization, often presented as a score or rating based on predefined metrics.  
   - **Contextual Example**: Companies use Data Quality Scorecards to track metrics like completeness, accuracy, and consistency, helping them identify areas for improvement.

3. **Metadata Schema**  
   - **Definition**: A comprehensive document describing the structure of metadata within an organization, detailing data elements, relat

CrewOutput(raw="1. **Data Governance Framework**  \n   - **Definition**: A structured approach to managing data policies, standards, and processes within an organization to ensure compliance, consistency, and quality of data assets.  \n   - **Contextual Example**: Organizations like XYZ utilize Data Governance Frameworks to implement data quality policies that prevent errors and ensure data usability across departments.\n\n2. **Data Quality Scorecard**  \n   - **Definition**: A tool used to monitor and evaluate the quality of data within an organization, often presented as a score or rating based on predefined metrics.  \n   - **Contextual Example**: Companies use Data Quality Scorecards to track metrics like completeness, accuracy, and consistency, helping them identify areas for improvement.\n\n3. **Metadata Schema**  \n   - **Definition**: A comprehensive document describing the structure of metadata within an organization, detailing data elements, relationships, and access controls